In [ ]:
import pandas as pd

# Load the CSV file into a pandas DataFrame
df_apps = pd.read_csv('data/apps_sample.csv')
df_transactions = pd.read_csv('data/txs_sample.csv')
# Specify the column name you want to analyze

# Get the number of unique entries for the specified column
unique_count_transactions = df_transactions['email'].nunique()
unique_count_apps = df_apps['email'].nunique()
print(f"The number of unique entries in the column '{'email'}' (txs_sample.csv): {unique_count_transactions}")
print(f"The number of unique entries in the column '{'email'}' (apps_sample.csv): {unique_count_apps}")


The number of unique entries in the column 'email' (txs_sample.csv): 2927
The number of unique entries in the column 'email' (apps_sample.csv): 3000


In [25]:

# Get the count of each unique entry in the specified column
status_counts = df_apps['status'].value_counts()

print(f"Counts of each entry in the column '{'status'}':")
print(status_counts)

Counts of each entry in the column 'status':
status
Repaid          12322
AutoRejected     3021
Outstanding      1242
Expired           481
Cancelled         202
Open               14
Name: count, dtype: int64


In [28]:
# Extract all transactions for a given customer
customer_email = '0521dc2fb6f29f79f126409112e21a975e6ed9a5bcc881fbd434045aa5d4a759'
customer_txs = df_transactions[df_transactions['email'] == customer_email]
print(f"All transactions for the customer with email '{customer_email}':")
# Only print amount, balance, category, subcategory, and date columns
print(customer_txs[['amount', 'balance', 'category', 'subcategory', 'date']])

All transactions for the customer with email '0521dc2fb6f29f79f126409112e21a975e6ed9a5bcc881fbd434045aa5d4a759':
         amount  balance                       category  \
2262593   12.03   615.61            ['Food and Dining']   
2262594   38.77   695.11              ['Uncategorized']   
2262595   13.18   602.43  ['Travel and Transportation']   
2262596   50.11   642.85            ['Food and Dining']   
2262597   17.96   733.88  ['Travel and Transportation']   
...         ...      ...                            ...   
2263495    1.92   134.16            ['Food and Dining']   
2263496    4.61   136.08            ['Food and Dining']   
2263497   20.00   140.69     ['Miscellaneous Expenses']   
2263498    7.44   160.69            ['Food and Dining']   
2263499   23.55   168.13            ['Food and Dining']   

                       subcategory        date  
2262593           ['Coffee Shops']  2024/08/30  
2262594          ['Uncategorized']  2024/08/30  
2262595  ['Ground Transportatio